#### Monday, December 23, 2024

This all runs in one pass.

In [1]:
# Deliberately set the OPENAI_API_KEY to an invalid value to ensure that the code is not using it.
import os
os.environ['OPENAI_API_KEY'] = "Nope!"

In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)
import pickle

In [3]:
examples = [
    {
        "question": "What is the capital of France?",
        "answer": "Paris",
    },
    {
        "question": "What is the capital of Spain?",
        "answer": "Madrid",
    },
    {
        "question": "What is the capital of Germany?",
        "answer": "Berlin",
    },
    {
        "question": "What is the capital of England?",
        "answer": "London",
    },
]

In [4]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

In [5]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: What is the capital of France?
AI: Paris
Human: What is the capital of Spain?
AI: Madrid
Human: What is the capital of Germany?
AI: Berlin
Human: What is the capital of England?
AI: London


In [6]:
from langchain_core.output_parsers import StrOutputParser

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a responsible for answering questions about countries. Only return the country name.",
        ),
        few_shot_prompt,
        ("human", "{question}"),
    ]
)

In [7]:
# model = ChatOpenAI()

from langchain_openai.chat_models import ChatOpenAI
import json
from os import getenv

# 2070 Super
api_key = "LMStudio"
lmstudio = "http://127.0.0.1:1234/v1"
model = "lmstudio-community/Qwen2.5-14B-Instruct-GGUF"

# chat = ChatOpenAI(base_url=lmstudio, model=model, api_key="LMStudio",  model_kwargs={"response_format": {"type": "json_object"}})
model = ChatOpenAI(base_url=lmstudio, api_key=api_key)

In [8]:
# Creating the LCEL chain with the prompt, model and a StrOutputParser()
chain = final_prompt | model | StrOutputParser()

In [9]:
result = chain.invoke(
    {
        "question": "What is the capital of America?"
    }
)

print(result)

Washington, D.C.


In [10]:
few_shot_prompt.examples

[{'question': 'What is the capital of France?', 'answer': 'Paris'},
 {'question': 'What is the capital of Spain?', 'answer': 'Madrid'},
 {'question': 'What is the capital of Germany?', 'answer': 'Berlin'},
 {'question': 'What is the capital of England?', 'answer': 'London'}]

In [11]:
# Pickle it:
with open('few_shot_prompt.pickle', 'wb') as f:
    pickle.dump(few_shot_prompt, f)

# Load it:
with open('few_shot_prompt.pickle', 'rb') as f:
    few_shot_prompt = pickle.load(f)